In [39]:
import numpy as np
import tqdm

In [40]:
nS = 500*100
nA = 7
N = np.zeros((nS, nA, nS), dtype=np.uint8)
Nsa = np.zeros((nS, nA), dtype=np.uint16)
Rho = np.zeros((nS, nA))
U = np.zeros((nS))

In [41]:
# load csv files one line at a time
fpath = './data/medium.csv'
with open(fpath, 'r') as f:
    for line in f:
        # skip header
        if line.startswith('s,a,r,sp'):
            continue
        s, a, r, sp = line.split(',')
        N[int(s)-1, int(a)-1, int(sp)-1] += 1
        Rho[int(s)-1, int(a)-1] += int(r)

In [42]:
Nsa = N.sum(axis=2)

In [51]:
# Estimate of reward function R(s,a) ~ Rho(s,a) / N(s,a)
R = np.zeros((nS, nA))
for s in range(nS):
    for a in range(nA):
        if Nsa[s, a] > 0:
            R[s, a] = Rho[s, a] / Nsa[s, a]

In [52]:
# Calculate action-value function Q(s,a) (non-vectorized)
Q = np.zeros((nS, nA))
gamma = 0.9
for i in tqdm.tqdm(range(10)):
    for s in range(nS):
        for a in range(nA):
            if Nsa[s, a] > 0:
                T = N[s, a] / Nsa[s, a]
                Q[s, a] = R[s, a] + gamma * T.dot(U)
    U = Q.max(axis=1)

100%|██████████| 10/10 [00:42<00:00,  4.25s/it]


In [38]:
# output the policy based on Q
fpath = './data/medium.policy'
policy = np.argmax(Q, axis=1)
with open(fpath, 'w') as f:
    for s in range(nS):
        f.write(str(policy[s]+1) + '\n')